# ETL VitalDB

### En esta segunda versión de notebook, realizo las correcciones a la selección de variables sugeridas en la reunión con los médicos anestesiólogos.

He descargado las tablas correspondientes a la base de datos VitalDB, de Physionet. **VitalDB** (Vital Signs DataBase) es un conjunto de datos abierto creado específicamente para facilitar estudios de aprendizaje automático relacionados con la monitorización de signos vitales en pacientes quirúrgicos.

Este conjunto de datos contiene información multiparamétrica de alta resolución de 6,388 casos, incluyendo 486,451 registros de datos numéricos y de formas de onda de 196 parámetros de monitorización intraoperatoria, 73 parámetros clínicos perioperatorios y 34 parámetros de resultados de laboratorio en series de tiempo.

En resumen, el conjunto de datos tiene las siguientes características:

* El conjunto de datos consta de datos de signos vitales intraoperatorios e información clínica perioperatoria de 6,388 casos.
* Los datos de signos vitales incluyen hasta 12 pistas de formas de onda y 184 pistas de datos numéricos obtenidos de múltiples dispositivos de anestesia aplicados a los pacientes durante la cirugía. El número total de pistas de datos es 486,451 (con un promedio de 87 y un rango de 16 a 129).
* Los datos de signos vitales tienen varios intervalos de tiempo según los dispositivos de anestesia, con una resolución de tiempo de 1 a 7 segundos para los datos numéricos y de 62.5 a 500 Hz para los datos de formas de onda. Cada archivo de caso contiene un promedio de 2.8 millones de puntos de datos.
* Los datos no están preprocesados porque el ruido del mundo real en los datos de signos vitales es esencial para el desarrollo de algoritmos de monitorización prácticos.
* Se proporcionan un total de 74 parámetros de información clínica perioperatoria y 34 resultados de laboratorio perioperatorios en series de tiempo para ayudar a interpretar la relación con los signos vitales intraoperatorios.

In [95]:
# Importar librerías necesarias
import vitaldb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import warnings
warnings.filterwarnings('ignore')

### Observación exploratoria inicial de la base de datos

El archivo vital es un archivo binario, grabado con el programa Vital Recorder, que contiene registros de series de tiempo de signos vitales. La especificación del formato del archivo vital se detalla en un documento dentro del repositorio de datos abierto.

Un paquete de Python llamado "vitaldb", que ayuda a leer y escribir archivos vitales, está disponible de forma gratuita en PyPI, el índice de paquetes de Python.

Después de instalar las librerías necesarias empiezo a explorar el contenido de mis carpetas.

In [96]:
# Configurar ruta donde tengo los archivos VitalDB
DATA_PATH = "/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0"
# Verificar que la ruta existe
if os.path.exists(DATA_PATH):
    print(f"Directorio encontrado: {DATA_PATH}")
    
    # Ver qué hay en el directorio principal
    contents = os.listdir(DATA_PATH)
    print(f"Contenido del directorio:")
    for item in contents:
        item_path = os.path.join(DATA_PATH, item)
        if os.path.isdir(item_path):
            print(f"  {item}/")
        else:
            print(f"  {item}")
    
    # Buscar archivos .vital en el directorio principal
    vital_files = [f for f in contents if f.endswith('.vital')]
    print(f"\nArchivos .vital encontrados en directorio principal: {len(vital_files)}")
    
    # Si no hay archivos .vital en el directorio principal, buscar en subdirectorios
    if len(vital_files) == 0:
        print("\nBuscando archivos .vital en subdirectorios...")
        for item in contents:
            item_path = os.path.join(DATA_PATH, item)
            if os.path.isdir(item_path):
                subdir_files = [f for f in os.listdir(item_path) if f.endswith('.vital')]
                if len(subdir_files) > 0:
                    print(f"  {item}/: {len(subdir_files)} archivos .vital")
    
else:
    print("Directorio no encontrado. Verificar ruta.")

Directorio encontrado: /Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0
Contenido del directorio:
  .DS_Store
  lab_parameters.csv
  track_names.csv
  clinical_parameters.csv
  lab_data.csv
  vital_files/
  clinical_data.csv
  LICENSE.txt
  SHA256SUMS.txt

Archivos .vital encontrados en directorio principal: 0

Buscando archivos .vital en subdirectorios...
  vital_files/: 6388 archivos .vital


In [97]:
# Actualizar ruta a la carpeta que contiene los archivos .vital
VITAL_FILES_PATH = "/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0/vital_files"
CSV_PATH = "/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0"

# Verificar archivos .vital
vital_files = [f for f in os.listdir(VITAL_FILES_PATH) if f.endswith('.vital')]
print(f"Total de archivos .vital: {len(vital_files)}")
print(f"Primeros 5 archivos: {vital_files[:5]}")


Total de archivos .vital: 6388
Primeros 5 archivos: ['4388.vital', '3698.vital', '3249.vital', '4759.vital', '1939.vital']


El archivo `clinical_data` proporciona datos perioperatorios relacionados con el paciente para ayudar a interpretar los datos de las bioseñales. Este archivo consta de un **ID de caso** y un **ID de paciente**, además de 72 parámetros clínicos que incluyen información del archivo del caso, datos demográficos, resultados, datos de laboratorio preoperatorios y datos relacionados con la cirugía y la anestesia.

In [98]:
#importo las tablas clinical_data y clinical_parameters
clinical_data_df = pd.read_csv('/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0/clinical_data.csv')
clinical_parameters_df = pd.read_csv('/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0/clinical_parameters.csv')

In [99]:
clinical_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6388 entries, 0 to 6387
Data columns (total 74 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   caseid               6388 non-null   int64  
 1   subjectid            6388 non-null   int64  
 2   casestart            6388 non-null   int64  
 3   caseend              6388 non-null   int64  
 4   anestart             6388 non-null   int64  
 5   aneend               6388 non-null   float64
 6   opstart              6388 non-null   int64  
 7   opend                6388 non-null   int64  
 8   adm                  6388 non-null   int64  
 9   dis                  6388 non-null   int64  
 10  icu_days             6388 non-null   int64  
 11  death_inhosp         6388 non-null   int64  
 12  age                  6388 non-null   object 
 13  sex                  6388 non-null   object 
 14  height               6388 non-null   float64
 15  weight               6388 non-null   f

In [100]:
clinical_parameters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Parameter    81 non-null     object
 1   Data Source  81 non-null     object
 2   Description  81 non-null     object
 3   Unit         53 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


### Selección de operaciones torácicas con VUP

Observando la columna `department` puedo extraer únicamente los registros que corresponden a cirugías torácicas.

In [117]:
cirugias_toracicas=clinical_data_df[clinical_data_df['department']=='Thoracic surgery']
cirugias_toracicas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1111 entries, 6 to 6375
Data columns (total 74 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   caseid               1111 non-null   int64  
 1   subjectid            1111 non-null   int64  
 2   casestart            1111 non-null   int64  
 3   caseend              1111 non-null   int64  
 4   anestart             1111 non-null   int64  
 5   aneend               1111 non-null   float64
 6   opstart              1111 non-null   int64  
 7   opend                1111 non-null   int64  
 8   adm                  1111 non-null   int64  
 9   dis                  1111 non-null   int64  
 10  icu_days             1111 non-null   int64  
 11  death_inhosp         1111 non-null   int64  
 12  age                  1111 non-null   object 
 13  sex                  1111 non-null   object 
 14  height               1111 non-null   float64
 15  weight               1111 non-null   float6

También es posible seleccionar sólo las operaciones con Ventilación unipulmonar y el parámetro clave  es `dltubesize`.

Este parámetro, que se encuentra en la columna "Parameter", tiene la descripción "Double lumen tube size" (tamaño del tubo de doble luz) y la unidad "Fr" (French). Un tubo de doble luz se utiliza específicamente para la ventilación unipulmonar.

In [118]:
# Suponiendo que tu DataFrame se llama clinical_parameters_df
descripcion = clinical_parameters_df.loc[clinical_parameters_df['Parameter'] == 'dltubesize', 'Description'].iloc[0]

print(descripcion)

Double lumen tube size


Para filtrar los casos que utilizaron ventilación unipulmonar, voy a extraer las filas donde la columna dltubesize contenga un valor. Si el valor es nulo (NaN), significa que no se usó un tubo de doble luz, por lo que no hubo ventilación unipulmonar. Si el valor no es nulo, es una fuerte indicación de que se utilizó SLV.

In [119]:
cirugias_VUP = cirugias_toracicas[cirugias_toracicas['dltubesize'].notna()]
cirugias_VUP.info()

<class 'pandas.core.frame.DataFrame'>
Index: 910 entries, 6 to 6375
Data columns (total 74 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   caseid               910 non-null    int64  
 1   subjectid            910 non-null    int64  
 2   casestart            910 non-null    int64  
 3   caseend              910 non-null    int64  
 4   anestart             910 non-null    int64  
 5   aneend               910 non-null    float64
 6   opstart              910 non-null    int64  
 7   opend                910 non-null    int64  
 8   adm                  910 non-null    int64  
 9   dis                  910 non-null    int64  
 10  icu_days             910 non-null    int64  
 11  death_inhosp         910 non-null    int64  
 12  age                  910 non-null    object 
 13  sex                  910 non-null    object 
 14  height               910 non-null    float64
 15  weight               910 non-null    float64

Encuentro que de los 6388 casos almacenados en la base de datos, 1111 son cirugías torácicas. Y de estas, 910 utilizaron tubo de doble luz, es decir, se realizaron con ventilación uni pulmonar (VUP).
De esta manera llego a tener las cirugías con las que voy a trabajar: **Cirugías Torácicas con Ventilación Unipulmonar**

### 3. Análisis de características demográficas. 

Conteo de pacientes según edad, sexo, ASA, peso, estatura, BMI.

PD3: El ASA Physical Status Classification System (o estado físico ASA) es una escala desarrollada por la American Society of Anesthesiologists (ASA) para clasificar el estado físico general de un paciente antes de una cirugía o procedimiento anestésico. Sirve para que anestesiólogos y cirujanos evalúen qué tan riesgosa puede ser la anestesia y la cirugía según las condiciones de salud del paciente, independientemente del tipo de cirugía.

Escala ASA Physical Status
- ASA I – Paciente sano, sin enfermedad sistémica, no fumador, sin consumo de alcohol o drogas.
- ASA II – Paciente con enfermedad sistémica leve (p. ej., hipertensión controlada, diabetes bien controlada, fumador social).
- ASA III – Paciente con enfermedad sistémica grave que limita su actividad pero no lo incapacita por completo (p. ej., angina estable, EPOC moderada, insuficiencia cardíaca controlada).
- ASA IV – Paciente con enfermedad sistémica grave que amenaza la vida (p. ej., insuficiencia cardíaca descompensada, sepsis grave).
- ASA V – Paciente moribundo, que probablemente no sobreviva 24 horas con o sin cirugía (p. ej., aneurisma roto, trauma masivo).
- ASA VI – Paciente con muerte cerebral que será sometido a extracción de órganos para trasplante.
- Además, se añade la letra E (Emergency) si la cirugía es de urgencia.Ejemplo: “ASA IIIE” = paciente con enfermedad sistémica grave, en cirugía de emergencia.

In [120]:
# 1. Convertir la columna 'age' a tipo numérico, forzando errores a NaN
cirugias_VUP['age'] = pd.to_numeric(cirugias_VUP['age'], errors='coerce')

In [121]:
# Análisis de características demográficas

def analyze_demographics(cirugias_VUP):
    
    print("\n=== ANÁLISIS DEMOGRÁFICO ===")
    
    # Distribución por edad
    print(f"\nEstadísticas de edad:")
    print(cirugias_VUP['age'].describe())
    
    # Distribución por sexo
    print(f"\nDistribución por sexo:")
    sex_dist = cirugias_VUP['sex'].value_counts()
    print(sex_dist)
    print(f"Porcentaje masculino: {sex_dist.get('M', 0)/len(cirugias_VUP)*100:.1f}%")
    
    # ASA Physical Status
    print(f"\nDistribución ASA-PS:")
    asa_dist = cirugias_VUP['asa'].value_counts().sort_index()
    for asa, count in asa_dist.items():
        print(f"  ASA {asa}: {count:,} ({count/len(cirugias_VUP)*100:.1f}%)")
    
    # Peso
    print(f"\nEstadísticas de peso:")
    print(cirugias_VUP['weight'].describe())

    # Estatura
    print(f"\nEstadísticas de estatura:")
    print(cirugias_VUP['height'].describe())    

    # BMI
    print(f"\nEstadísticas de BMI:")
    print(cirugias_VUP['bmi'].describe())    

In [122]:
analyze_demographics(cirugias_VUP)


=== ANÁLISIS DEMOGRÁFICO ===

Estadísticas de edad:
count    910.000000
mean      59.316484
std       13.813183
min       16.000000
25%       52.000000
50%       61.000000
75%       69.000000
max       89.000000
Name: age, dtype: float64

Distribución por sexo:
sex
M    505
F    405
Name: count, dtype: int64
Porcentaje masculino: 55.5%

Distribución ASA-PS:
  ASA 1.0: 162 (17.8%)
  ASA 2.0: 612 (67.3%)
  ASA 3.0: 97 (10.7%)
  ASA 4.0: 4 (0.4%)
  ASA 6.0: 9 (1.0%)

Estadísticas de peso:
count    910.000000
mean      62.011209
std       10.369091
min       34.000000
25%       54.400000
50%       61.100000
75%       68.700000
max       98.000000
Name: weight, dtype: float64

Estadísticas de estatura:
count    910.000000
mean     162.922527
std        8.563554
min      138.400000
25%      156.400000
50%      163.250000
75%      169.175000
max      186.000000
Name: height, dtype: float64

Estadísticas de BMI:
count    910.000000
mean      23.330110
std        3.244648
min       14.500000
2

De lo anterior puedo observar que:

* La mayoría de los individuos en la muestra son del **sexo masculino**, representando aproximadamente el 55.5% del total.
* El promedio de edad son  **59 años**, siendo la edad mínima 16 y la máxima 89 años.
* La mayoría de los pacientes se clasifican con un **estado físico ASA 2.0** (67.3%), lo que indica la presencia de una enfermedad sistémica leve que no limita la actividad.
* El **BMI promedio** es de 23.3, lo que se encuentra dentro del rango de peso normal. 

### Características de las cirugías

Voy a revisar los siguientes aspectos:
- Duración de la cirugía
- Duración de la anestesia
- Duración del caso
- Tipo de anestesia
- Approach
- Posición

In [123]:
# Calculo la duración de la operación restando tiempo de inicio y tiempo de finalización
# Creo la columna 'dur_op_seg'

if 'opstart' in cirugias_VUP.columns and 'opend' in cirugias_VUP.columns:
    cirugias_VUP['dur_op_seg'] = cirugias_VUP['opend'] - cirugias_VUP['opstart']
        
    print("Duración de cirugía (segundos):")
    print(cirugias_VUP['dur_op_seg'].describe())

# Calculo la duración de la anestesia restando tiempo de inicio y tiempo de finalización
# Creo la columna 'dur_anest_seg'

if 'anestart' in cirugias_VUP.columns and 'aneend' in cirugias_VUP.columns:
    cirugias_VUP['dur_anest_seg'] = cirugias_VUP['aneend'] - cirugias_VUP['anestart']
        
    print(f"\nDuración de anestesia (segundos):")
    print(cirugias_VUP['dur_anest_seg'].describe())    

# Calculo la duración de cada caso restando tiempo de inicio y tiempo de finalización
# Creo la columna 'dur_case_seg'

if 'casestart' in cirugias_VUP.columns and 'caseend' in cirugias_VUP.columns:
    cirugias_VUP['dur_case_seg'] = cirugias_VUP['caseend'] - cirugias_VUP['casestart']
        
    print(f"\nDuración de caso (segundos):")
    print(cirugias_VUP['dur_case_seg'].describe()) 

#Observo la distribución de las cirugías según el tipo de anestesia
print(f"\nTipo de anestesia:")
anest_types = cirugias_VUP['ane_type'].value_counts()
for atype, count in anest_types.items():
    print(f"  {atype}: {count:,} ({count/len(cirugias_VUP)*100:.1f}%)")

#Observo la distribución de las cirugías según approach
print(f"\nApproach:")
anest_types = cirugias_VUP['approach'].value_counts()
for atype, count in anest_types.items():
    print(f"  {atype}: {count:,} ({count/len(cirugias_VUP)*100:.1f}%)")    

#Observo la distribución de las cirugías según la posición
print(f"\nPosición:")
anest_types = cirugias_VUP['position'].value_counts()
for atype, count in anest_types.items():
    print(f"  {atype}: {count:,} ({count/len(cirugias_VUP)*100:.1f}%)")

Duración de cirugía (segundos):
count      910.000000
mean      7930.918681
std       5639.708936
min       1200.000000
25%       4500.000000
50%       6900.000000
75%       9657.000000
max      57300.000000
Name: dur_op_seg, dtype: float64

Duración de anestesia (segundos):
count      910.000000
mean     12371.142857
std       6249.969066
min       3480.000000
25%       8355.000000
50%      11640.000000
75%      14700.000000
max      65700.000000
Name: dur_anest_seg, dtype: float64

Duración de caso (segundos):
count      910.000000
mean     11811.790110
std       6200.132358
min       3159.000000
25%       7754.750000
50%      10831.500000
75%      14177.750000
max      62494.000000
Name: dur_case_seg, dtype: float64

Tipo de anestesia:
  General: 910 (100.0%)

Approach:
  Videoscopic: 771 (84.7%)
  Open: 117 (12.9%)
  Robotic: 22 (2.4%)

Posición:
  Left lateral decubitus: 472 (51.9%)
  Right lateral decubitus: 338 (37.1%)
  Supine: 87 (9.6%)
  Sitting: 3 (0.3%)
  Prone: 1 (0.1%)


### Limpieza de datos tabla cirugías VUP - Valores nulos

In [125]:
# 1. INFORMACIÓN BÁSICA
print("\n1. INFORMACIÓN BÁSICA")
print("-" * 30)
print(f"Shape total: {cirugias_VUP.shape}")
print(f"Pacientes únicos: {cirugias_VUP['subjectid'].nunique()}")
print(f"Cirugías únicas: {cirugias_VUP['caseid'].nunique()}")


1. INFORMACIÓN BÁSICA
------------------------------
Shape total: (910, 77)
Pacientes únicos: 882
Cirugías únicas: 910


In [126]:
# 2. VALORES FALTANTES
print("\n2. ANÁLISIS DE VALORES FALTANTES")
print("-" * 40)
missing_analysis = cirugias_VUP.isnull().sum()
missing_pct = (missing_analysis / len(cirugias_VUP)) * 100
missing_df = pd.DataFrame({
        'Missing_Count': missing_analysis,
        'Missing_Pct': missing_pct
    }).sort_values('Missing_Pct', ascending=False)
    
print("Columnas con valores faltantes:")
print(missing_df[missing_df['Missing_Count'] > 0])


2. ANÁLISIS DE VALORES FALTANTES
----------------------------------------
Columnas con valores faltantes:
                     Missing_Count  Missing_Pct
lmasize                        910   100.000000
cline2                         904    99.340659
aline2                         900    98.901099
tubesize                       881    96.813187
preop_sao2                     827    90.879121
preop_paco2                    827    90.879121
preop_pao2                     827    90.879121
preop_be                       827    90.879121
preop_hco3                     827    90.879121
preop_ph                       825    90.659341
iv2                            579    63.626374
intraop_ebl                    393    43.186813
cline1                         322    35.384615
intraop_uo                     233    25.604396
preop_na                       165    18.131868
preop_k                        164    18.021978
preop_pt                       152    16.703297
preop_aptt                   

Voy a revisar la tabla de parámetros clínicos para conocer la descripción de los parámetros que tienen datos faltantes

In [127]:
# 1. Obtener la lista de parámetros con valores faltantes
# La lista de parámetros con valores faltantes se encuentra en el índice
# del DataFrame `missing_df`.
missing_parameters = missing_df[missing_df['Missing_Count'] > 0].index.tolist()

# 3. Filtrar el DataFrame clinical_parameters para obtener las descripciones
# Se usa el método isin() para buscar cada uno de los parámetros de la lista
descriptions_of_missing_params = clinical_parameters_df[
    clinical_parameters_df['Parameter'].isin(missing_parameters)
]

# 4. Imprimir los resultados
print("\nDescripciones de los parámetros con valores faltantes:")
print("-" * 50)
print(descriptions_of_missing_params[['Parameter', 'Description']])


Descripciones de los parámetros con valores faltantes:
--------------------------------------------------
              Parameter                         Description
17                  asa  ASA Physical status classification
24             position                   Surgical position
37             preop_hb             Preoperative hemoglobin
38            preop_plt         Preoperative platelet count
39             preop_pt                     Preoperative PT
40           preop_aptt                   Preoperative aPTT
41             preop_na                     Preoperative Na
42              preop_k                      Preoperative K
43           preop_gluc                Preoperative glucose
44            preop_alb                Preoperative albumin
45            preop_ast                    Preoperative GOT
46            preop_alt                    Preoperative GPT
47            preop_bun    Preoperative blood urea nitrogen
48             preop_cr             Preoperative crea

Las que  tienen más del 90% de valores nulos son:

LMA size(lmasize)

cline2 (Site of central line (2))

aline2 (Site of arterial line (2))

tubesize (Endotracheal tube size)

preop_pao2 (Preoperative PaO2)

preop_be (Preoperative base excess)

preop_hco3 (Preoperative HCO3)

preop_sao2 (Preoperative SpO2)

preop_paco2 (Preoperative PaCO2)

preop_ph (Preoperative pH)

Decido eliminar estas columnas, porque unas tienen que ver con información que no existe debido al tipo de operación, y otras son valores pre-operatorios. Parámetros como preop_pao2, preop_paco2, preop_ph, preop_sao2, preop_hco3 y preop_be se obtienen de análisis de gases arteriales.  Este tipo de prueba es invasiva y no se realiza de forma rutinaria en todos los pacientes antes de la cirugía.

También están las siguientes con porcentajes de cantidad de nulos entre 63.6% y 12.8%

iv2 (Site of IV line (2))

intraop_ebl (Estimated blood loss)

cline1 (Site of central line (1))

intraop_uo (Intraoperative urine output)

preop_na (Preoperative Na)

preop_k (Preoperative K)

preop_pt (Preoperative PT)

preop_aptt (Preoperative aPTT)

preop_alt (Preoperative GPT)

preop_ast (Preoperative GOT)

preop_alb (Preoperative albumin)

preop_bun (Preoperative blood urea nitrogen)

preop_gluc (Preoperative glucose)

preop_cr (Preoperative creatinine)

preop_plt (Preoperative platelet count)

preop_hb (Preoperative hemoglobin)

Columnas a eliminar: ['lmasize', 'cline1', 'cline2', 'aline2', 'tubesize', 'iv2', 'preop_pao2', 'preop_be', 'preop_hco3', 'preop_sao2', 'preop_paco2', 'preop_ph', 'intraop_ebl', 'intraop_uo', 'preop_na', 'preop_k', 'preop_pt', 'preop_aptt', 'preop_alt', 'preop_ast', 'preop_alb', 'preop_bun', 'preop_gluc', 'preop_cr', 'preop_plt'].

Tomo esta decisión pues los porcentajes de datos nulos son muy elevados y/o porque además no son variables que vaya a utilizar en mi modelo de ML

In [128]:
columnas_a_eliminar = ['lmasize', 'cline1', 'cline2', 'aline2', 'tubesize', 'iv2', 'preop_pao2', 'preop_be',
'preop_hco3', 'preop_sao2', 'preop_paco2', 'preop_ph', 'intraop_ebl', 'intraop_uo', 'preop_na', 
'preop_k', 'preop_pt', 'preop_aptt', 'preop_alt', 'preop_ast', 'preop_alb', 'preop_bun', 
'preop_gluc', 'preop_cr', 'preop_plt']

# Crear un nuevo DataFrame con las columnas eliminadas
cirugias_VUP_filtrado = cirugias_VUP.drop(columns=columnas_a_eliminar)

# Verificar que el DataFrame original no ha cambiado
print(f"Número de columnas en el DataFrame original: {cirugias_VUP.shape[1]}")

# Verificar el nuevo DataFrame
print(f"Número de columnas en el nuevo DataFrame: {cirugias_VUP_filtrado.shape[1]}")

Número de columnas en el DataFrame original: 77
Número de columnas en el nuevo DataFrame: 52


In [129]:
#Vuelvo a consultar valores faltantes
# 2. VALORES FALTANTES
print("\n2. ANÁLISIS DE VALORES FALTANTES")
print("-" * 40)
missing_analysis = cirugias_VUP_filtrado.isnull().sum()
missing_pct = (missing_analysis / len(cirugias_VUP_filtrado)) * 100
missing_df = pd.DataFrame({
        'Missing_Count': missing_analysis,
        'Missing_Pct': missing_pct
    }).sort_values('Missing_Pct', ascending=False)
    
print("Columnas con valores faltantes:")
print(missing_df[missing_df['Missing_Count'] > 0])


2. ANÁLISIS DE VALORES FALTANTES
----------------------------------------
Columnas con valores faltantes:
                     Missing_Count  Missing_Pct
preop_hb                       117    12.857143
intraop_crystalloid             47     5.164835
cormack                         28     3.076923
asa                             26     2.857143
position                         9     0.989011
aline1                           4     0.439560
airway                           3     0.329670
iv1                              1     0.109890


De estas columnas me parece relevante conservarlas preop_hb, asa y position, así que voy a manejar los datos nulos con técnicas de imputación de datos.

Las demas columnas las elimino pues en la verificacion de variables realizada con los médicos anestesiólogos, se sugirió que no eran determinantes para la predicción de hipoxemia. También elimino columnas que, aunque no tienen datos nulos, también fueron identificadas como poco relevantes: 'subjectid', 'opstart', 'opend', 'adm', 'dis', 'icu_days', 'death_inhosp', 'optype', 'dx', 'opname', 'intraop_colloid'

In [130]:
columnas_a_eliminar2 = ['intraop_crystalloid', 'cormack', 'aline1', 'airway', 'iv1', 'subjectid', 
                        'opstart', 'opend', 'adm', 'dis', 'icu_days', 'death_inhosp', 'optype', 'dx', 'opname', 'intraop_colloid']

# Crear un nuevo DataFrame con las columnas eliminadas
cirugias_VUP_filtrado2 = cirugias_VUP_filtrado.drop(columns=columnas_a_eliminar2)

# Verificar que el DataFrame original no ha cambiado
print(f"Número de columnas en el DataFrame original: {cirugias_VUP_filtrado.shape[1]}")

# Verificar el nuevo DataFrame
print(f"Número de columnas en el nuevo DataFrame: {cirugias_VUP_filtrado2.shape[1]}")

Número de columnas en el DataFrame original: 52
Número de columnas en el nuevo DataFrame: 36


In [131]:
# Imputar la hemoglobina preoperatoria
mediana_hb = cirugias_VUP_filtrado2['preop_hb'].median()
cirugias_VUP_filtrado2['preop_hb'].fillna(mediana_hb, inplace=True)

Para las variables categóricas, `asa` y `position`, la mejor opción es la imputación por la moda (el valor más frecuente). Al reemplazar los valores nulos con la categoría que más se repite, estoy manteniendo la distribución original de la variable y evito introducir nuevos valores que no existen en el conjunto de datos.

In [132]:
# Definir la lista de variables categóricas a imputar
columnas_categoricas = ['asa', 'position']

for col in columnas_categoricas:
    moda_col = cirugias_VUP_filtrado2[col].mode()[0]
    cirugias_VUP_filtrado2[col].fillna(moda_col, inplace=True)

In [133]:
#Vuelvo a consultar valores faltantes
# 2. VALORES FALTANTES
print("\n2. ANÁLISIS DE VALORES FALTANTES")
print("-" * 40)
missing_analysis = cirugias_VUP_filtrado2.isnull().sum()
missing_pct = (missing_analysis / len(cirugias_VUP_filtrado2)) * 100
missing_df = pd.DataFrame({
        'Missing_Count': missing_analysis,
        'Missing_Pct': missing_pct
    }).sort_values('Missing_Pct', ascending=False)
    
print("Columnas con valores faltantes:")
print(missing_df[missing_df['Missing_Count'] > 0])


2. ANÁLISIS DE VALORES FALTANTES
----------------------------------------
Columnas con valores faltantes:
Empty DataFrame
Columns: [Missing_Count, Missing_Pct]
Index: []


### Limpieza de datos tabla cirugías VUP - Registros duplicados

In [134]:
# ANÁLISIS DE REGISTROS DUPLICADOS
# Duplicados exactos
print("\n3. ANÁLISIS DE REGISTROS DUPLICADOS")
print("-" * 40)
exact_duplicates = cirugias_VUP_filtrado2.duplicated().sum()
print(f"Registros duplicados exactos: {exact_duplicates:,}")


3. ANÁLISIS DE REGISTROS DUPLICADOS
----------------------------------------
Registros duplicados exactos: 0


In [135]:
print("\n1. INFORMACIÓN BÁSICA DE LOS DATOS SIN NULOS NI REPETIDOS")
print("-" * 30)
print(f"Shape total: {cirugias_VUP_filtrado2.shape}")
print(f"Cirugías únicas: {cirugias_VUP_filtrado2['caseid'].nunique()}")


1. INFORMACIÓN BÁSICA DE LOS DATOS SIN NULOS NI REPETIDOS
------------------------------
Shape total: (910, 36)
Cirugías únicas: 910


Tengo una tabla que contiene 36 parámetros pre-operatorios de 910 cirugías. No hay datos nulos ni registros duplicados.

In [136]:
cirugias_VUP_filtrado2.head()

,caseid,casestart,caseend,anestart,aneend,age,sex,height,weight,bmi,...,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca,dur_op_seg,dur_anest_seg,dur_case_seg
6,7,0,15770,477,14817.0,52,F,167.7,62.3,22.2,...,0,120,0,0,0,0,0,11400,14340.0,15770
12,13,0,10811,-703,10697.0,67,F,153.0,64.9,27.7,...,0,95,0,0,0,0,0,6180,11400.0,10811
19,20,0,26476,86,27086.0,75,M,173.6,61.3,20.3,...,0,180,0,5,250,0,600,22489,27000.0,26476
25,26,0,10711,-518,10282.0,64,M,169.7,62.6,21.7,...,0,120,0,0,0,0,0,6600,10800.0,10711
27,28,0,26902,-258,27162.0,67,F,148.9,49.9,22.5,...,0,170,0,0,400,0,0,22800,27420.0,26902


### Exploracion de las tablas .vital

Voy a observar el primer archivo de mi tabla, cuyo caseid es 0007. Voy a ingresar manualmente el path en la función, para hacer una observación preliminar. Quiero extraer las siguientes variables:'spo2', 'etco2', 'fio2', 'vt', 'pip', 'hr', 'rr'.

Observando la tabla de `track_names`encuentro que lo nombres de estas variables en las tablas de vitals son los siguientes:

'Solar8000/ETCO2','Solar8000/FEO2','Solar8000/FIO2','Solar8000/HR','Solar8000/INCO2','Solar8000/RR','Solar8000/RR_CO2','Solar8000/VENT_RR','Solar8000/PLETH_SPO2','Solar8000/PLETH_HR','Solar8000/VENT_PIP','Solar8000/VENT_TV'

También, por recomendaciónde los médicos anestesiólogos, quiero observar MAC (Minimum alveolar concentration of volatile), CI (Cardiac Index) y CO (Cardiac Output):

'Primus/MAC','Vigileo/CI','Vigileo/CO','EV1000/CI','EV1000/CO','CardioQ/CI','CardioQ/CO','Vigilance/CI','Vigilance/CO'

In [67]:
#El caseid de la primera cirugía que quiero observar es 7. Los nombres de los archivos tienen 4 caracteres
# es decir el nombre de este archivo es 0007.vital
archivo_cargar = '/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0/vital_files/0007.vital'
parametros = ['Solar8000/ETCO2','Solar8000/FEO2','Solar8000/FIO2','Solar8000/HR','Solar8000/INCO2','Solar8000/RR','Solar8000/RR_CO2','Solar8000/VENT_RR','Solar8000/PLETH_SPO2','Solar8000/PLETH_HR','Solar8000/VENT_PIP','Solar8000/VENT_TV','Primus/MAC','Vigileo/CI','Vigileo/CO','EV1000/CI','EV1000/CO','CardioQ/CI','CardioQ/CO','Vigilance/CI','Vigilance/CO']

# 2. Llamo a la función VitalFile y asigno el resultado a una variable
#tomo un intervalo de 10 seg. Intenté con 1 pero me devolvía muchos datos nulos. Luego intenté con 10 y 20seg
# y la cantidad de datos nulos era la misma, entonces me quedo con el valor de 10seg para el intervalo
datos_de_cirugia = vitaldb.VitalFile(archivo_cargar).to_pandas (parametros, interval = 10)

# 3. Ahora puedo trabajar con la variable 'datos_de_cirugia'
datos_de_cirugia.head()

,Solar8000/ETCO2,Solar8000/FEO2,Solar8000/FIO2,Solar8000/HR,Solar8000/INCO2,Solar8000/RR,Solar8000/RR_CO2,Solar8000/VENT_RR,Solar8000/PLETH_SPO2,Solar8000/PLETH_HR,...,Solar8000/VENT_TV,Primus/MAC,Vigileo/CI,Vigileo/CO,EV1000/CI,EV1000/CO,CardioQ/CI,CardioQ/CO,Vigilance/CI,Vigilance/CO
0,1.0,21.0,21.0,65.0,1.0,NaN,NaN,NaN,61.0,99.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,21.0,21.0,68.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,21.0,21.0,67.0,1.0,NaN,NaN,NaN,65.0,101.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,21.0,21.0,67.0,1.0,NaN,NaN,NaN,65.0,101.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,21.0,21.0,67.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
datos_de_cirugia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1577 entries, 0 to 1576
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Solar8000/ETCO2       1514 non-null   float64
 1   Solar8000/FEO2        1514 non-null   float64
 2   Solar8000/FIO2        1514 non-null   float64
 3   Solar8000/HR          1467 non-null   float64
 4   Solar8000/INCO2       1514 non-null   float64
 5   Solar8000/RR          0 non-null      float64
 6   Solar8000/RR_CO2      1395 non-null   float64
 7   Solar8000/VENT_RR     1397 non-null   float64
 8   Solar8000/PLETH_SPO2  1518 non-null   float64
 9   Solar8000/PLETH_HR    1518 non-null   float64
 10  Solar8000/VENT_PIP    1359 non-null   float64
 11  Solar8000/VENT_TV     1390 non-null   float64
 12  Primus/MAC            1510 non-null   float64
 13  Vigileo/CI            0 non-null      float64
 14  Vigileo/CO            0 non-null      float64
 15  EV1000/CI            

In [138]:
# Estadísticas de SPO2
print(f"\nEstadísticas de SPO2:")
print(datos_de_cirugia['Solar8000/PLETH_SPO2'].describe())


Estadísticas de SPO2:
count    1518.000000
mean       99.603426
std         3.341994
min        54.000000
25%       100.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: Solar8000/PLETH_SPO2, dtype: float64


### NOTA
En este primer proceso de importación traje información de diferentes variables que contenía información de RR (Respiratory Rate), CI(Cardiac Index) y CO (Cardiac Output). 

Encontré que la variable `Solar8000/RR`, está casi totlamente vacía. En la tabla de nombres dice que esta variable corresponde a Frecuencia Respiratoria basada en el ECG. Hay otras dos RR: `Solar8000/RR_CO2`(Respiratory rate based on capnography) y `Solar8000/VENT_RR` (Respiratory rate from ventilator). Voy a eliminar la que está vacía y me quedo con las otras dos.

Tambien veo que todas las que observé para CI y CO están completamente vacías, entonces, a pesar de que estos dos datos sería desable tenerlos, debo eliminar las columnas pues ninguna me aporta información.

Voy a volver a importar los datos del caso 0007, con los cambios de las variables para observar de nuevo la información que puedo obetener de este caso

In [141]:
# El path del archivo es el mismo. Los parámetros son los nuevos, quitando Solar8000/RR y 
# trayendo Solar8000/RR_CO2 y Solar8000/VENT_RR
archivo_cargar = '/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0/vital_files/0007.vital'
parametros = ['Solar8000/RR_CO2','Solar8000/ETCO2','Solar8000/FEO2','Solar8000/FIO2','Solar8000/HR','Solar8000/INCO2','Solar8000/PLETH_HR','Solar8000/PLETH_SPO2','Solar8000/VENT_PIP','Solar8000/VENT_RR','Solar8000/VENT_TV','Primus/MAC']

datos_de_cirugia = vitaldb.VitalFile(archivo_cargar).to_pandas (parametros, interval = 10)

datos_de_cirugia.head()

,Solar8000/RR_CO2,Solar8000/ETCO2,Solar8000/FEO2,Solar8000/FIO2,Solar8000/HR,Solar8000/INCO2,Solar8000/PLETH_HR,Solar8000/PLETH_SPO2,Solar8000/VENT_PIP,Solar8000/VENT_RR,Solar8000/VENT_TV,Primus/MAC
0,NaN,1.0,21.0,21.0,65.0,1.0,99.0,61.0,NaN,NaN,NaN,0.0
1,NaN,1.0,21.0,21.0,68.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,1.0,21.0,21.0,67.0,1.0,101.0,65.0,NaN,NaN,NaN,0.0
3,NaN,1.0,21.0,21.0,67.0,1.0,101.0,65.0,NaN,NaN,NaN,0.0
4,NaN,1.0,21.0,21.0,67.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0


En las primeras filas veo varias columnas con solo Nan, entonces las voy a observar

In [142]:
datos_de_cirugia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1577 entries, 0 to 1576
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Solar8000/RR_CO2      1395 non-null   float32
 1   Solar8000/ETCO2       1514 non-null   float32
 2   Solar8000/FEO2        1514 non-null   float32
 3   Solar8000/FIO2        1514 non-null   float32
 4   Solar8000/HR          1467 non-null   float32
 5   Solar8000/INCO2       1514 non-null   float32
 6   Solar8000/PLETH_HR    1518 non-null   float32
 7   Solar8000/PLETH_SPO2  1518 non-null   float32
 8   Solar8000/VENT_PIP    1359 non-null   float32
 9   Solar8000/VENT_RR     1397 non-null   float32
 10  Solar8000/VENT_TV     1390 non-null   float32
 11  Primus/MAC            1510 non-null   float32
dtypes: float32(12)
memory usage: 74.1 KB


A continuación realizo el tratamiento de datos nulos y registros duplicados

In [143]:
#Quiero calcular porcentajes valores faltantes
# 2. VALORES FALTANTES
print("\n2. ANÁLISIS DE VALORES FALTANTES")
print("-" * 40)
missing_analysis2 = datos_de_cirugia.isnull().sum()
missing_pct2 = (missing_analysis2 / len(datos_de_cirugia)) * 100
missing_df2 = pd.DataFrame({
        'Missing_Count': missing_analysis2,
        'Missing_Pct': missing_pct2
    }).sort_values('Missing_Pct', ascending=False)
    
print("Columnas con valores faltantes:")
print(missing_df2[missing_df2['Missing_Count'] > 0])


2. ANÁLISIS DE VALORES FALTANTES
----------------------------------------
Columnas con valores faltantes:
                      Missing_Count  Missing_Pct
Solar8000/VENT_PIP              218    13.823716
Solar8000/VENT_TV               187    11.857958
Solar8000/RR_CO2                182    11.540900
Solar8000/VENT_RR               180    11.414077
Solar8000/HR                    110     6.975269
Primus/MAC                       67     4.248573
Solar8000/ETCO2                  63     3.994927
Solar8000/FEO2                   63     3.994927
Solar8000/FIO2                   63     3.994927
Solar8000/INCO2                  63     3.994927
Solar8000/PLETH_HR               59     3.741281
Solar8000/PLETH_SPO2             59     3.741281


Ya no veo un caso tan extremo de valores faltantes, la columna con más nulos tiene un 13.8%. Para no alterar la secuencia temporal de los datos, no debería eliminar registros, sino imputarlos con el método de último valor observado. Pruebo esto con la tabla exploratoria.

In [144]:
# Realizar la imputación
df_imputado = datos_de_cirugia.ffill()

# Rellenar los valores nulos al inicio de la serie
df_imputado_completo = df_imputado.ffill().bfill()

# Observar el resultado (las primeras filas con valores nulos imputados)
df_imputado_completo.head()

,Solar8000/RR_CO2,Solar8000/ETCO2,Solar8000/FEO2,Solar8000/FIO2,Solar8000/HR,Solar8000/INCO2,Solar8000/PLETH_HR,Solar8000/PLETH_SPO2,Solar8000/VENT_PIP,Solar8000/VENT_RR,Solar8000/VENT_TV,Primus/MAC
0,0.0,1.0,21.0,21.0,65.0,1.0,99.0,61.0,4.0,0.0,893.0,0.0
1,0.0,1.0,21.0,21.0,68.0,1.0,99.0,61.0,4.0,0.0,893.0,0.0
2,0.0,1.0,21.0,21.0,67.0,1.0,101.0,65.0,4.0,0.0,893.0,0.0
3,0.0,1.0,21.0,21.0,67.0,1.0,101.0,65.0,4.0,0.0,893.0,0.0
4,0.0,1.0,21.0,21.0,67.0,1.0,101.0,65.0,4.0,0.0,893.0,0.0


In [145]:
df_imputado_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1577 entries, 0 to 1576
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Solar8000/RR_CO2      1577 non-null   float32
 1   Solar8000/ETCO2       1577 non-null   float32
 2   Solar8000/FEO2        1577 non-null   float32
 3   Solar8000/FIO2        1577 non-null   float32
 4   Solar8000/HR          1577 non-null   float32
 5   Solar8000/INCO2       1577 non-null   float32
 6   Solar8000/PLETH_HR    1577 non-null   float32
 7   Solar8000/PLETH_SPO2  1577 non-null   float32
 8   Solar8000/VENT_PIP    1577 non-null   float32
 9   Solar8000/VENT_RR     1577 non-null   float32
 10  Solar8000/VENT_TV     1577 non-null   float32
 11  Primus/MAC            1577 non-null   float32
dtypes: float32(12)
memory usage: 74.1 KB


Veo que funciona. Entonces, teniendo en cuenta las variables definitivas, según los aprendizajes obtenidos del proceso realizado con un sólo archivo, voy a importar todos los archivos correspondientes a los caseid de las cirugías torácicas con VUP 

Ya probada la funcion por medio de la cual se tiene acceso a los datos de los archivos vital, voy a procesar los archivos que corresponden a las 910 cirugías y consolidar esta información en una sola tabla de datos.

### Pasos a Seguir

1. Extraer los caseid: Obtengo la lista de los caseid del DataFrame cirugias_VUP_filtrado.

2. Preparar el bucle: Defino una lista vacía para almacenar los DataFrames de cada cirugía y establezco la ruta base donde se encuentran los archivos .vital.

3. Iterar y cargar los datos: Dentro de un bucle for, recorro cada caseid. En cada iteración:

    - Formateo el caseid con ceros a la izquierda para que coincida con el nombre del archivo (ej. 7 a '0007').

    - Construyo la ruta completa del archivo .vital.

    - Utilizo la clase vitaldb.VitalFile() y su método .to_pandas() para cargar solo los parámetros que necesito.

    - Añado una nueva columna al DataFrame de esa cirugía con el caseid para poder identificar los datos de cada paciente.

    - Guardo el DataFrame de cada cirugía en la lista definida al inicio.

4. Combinar los datos: Después de que el bucle termine, uso pd.concat() para unir todos los DataFrames de la lista en una única y gran tabla.

In [ ]:
# 1. Definir la ruta base donde se encuentran los archivos .vital
vital_files_dir = '/Users/anap/Desktop/JAVERIANA/TGrado/DATA&SCRIPTS/vitaldb-1.0.0/vital_files/'

# 2. Extraer los caseid del DataFrame de cirugías filtradas
caseids_a_procesar = cirugias_VUP_filtrado2['caseid'].tolist()

# 3. Definir los parámetros (track names) que necesito
parametros = ['Solar8000/ETCO2','Solar8000/FEO2','Solar8000/FIO2',
              'Solar8000/HR','Solar8000/INCO2','Solar8000/RR_CO2',
              'Solar8000/PLETH_SPO2','Solar8000/PLETH_HR',
              'Solar8000/VENT_PIP','Solar8000/VENT_TV','Solar8000/VENT_RR',
              'Primus/MAC']


# Lista para almacenar los DataFrames de cada cirugía
all_vitals_data = []

# 4. Iterar sobre cada caseid, cargar los datos y guardarlos
for caseid in caseids_a_procesar:
    try:
        # Formatear el caseid a un string de 4 dígitos con ceros
        filename = f"{caseid:04d}.vital"
        file_path = os.path.join(vital_files_dir, filename)

        print(f"Procesando archivo: {filename}")

        # Crear el objeto VitalFile y cargar los datos
        #vital_file = vitaldb.VitalFile(file_path)
        
        # Cargar los datos a un DataFrame, utilizando un intervalo de 10 segundos
        df_temp = vitaldb.VitalFile(file_path).to_pandas(parametros, interval=10)

        # Añadir una columna para identificar el caseid en el DataFrame final
        df_temp['caseid'] = caseid
        
        # Añadir el DataFrame a la lista
        all_vitals_data.append(df_temp)

    except FileNotFoundError:
        print(f"Advertencia: Archivo {filename} no encontrado. Saltando este caso.")
    except Exception as e:
        print(f"Error procesando el archivo {filename}: {e}")

# 5. Concatenar todos los DataFrames en uno solo
if all_vitals_data:
    final_df = pd.concat(all_vitals_data, ignore_index=True)
    print("\nProcesamiento completado. El DataFrame final tiene la siguiente forma:")
    print(final_df.shape)
    final_df.head()
else:
    print("No se pudieron cargar archivos. El DataFrame final está vacío.")

Procesando archivo: 0007.vital
Procesando archivo: 0013.vital
Procesando archivo: 0020.vital
Procesando archivo: 0026.vital
Procesando archivo: 0028.vital
Procesando archivo: 0031.vital
Procesando archivo: 0038.vital
Procesando archivo: 0044.vital
Procesando archivo: 0046.vital
Procesando archivo: 0049.vital
Procesando archivo: 0067.vital
Procesando archivo: 0068.vital
Procesando archivo: 0074.vital
Procesando archivo: 0075.vital
Procesando archivo: 0077.vital
Procesando archivo: 0093.vital
Procesando archivo: 0103.vital
Procesando archivo: 0104.vital
Procesando archivo: 0113.vital
Procesando archivo: 0126.vital
Procesando archivo: 0128.vital
Procesando archivo: 0130.vital
Procesando archivo: 0132.vital
Procesando archivo: 0136.vital
Procesando archivo: 0139.vital
Procesando archivo: 0145.vital
Procesando archivo: 0149.vital
Procesando archivo: 0152.vital
Procesando archivo: 0153.vital
Procesando archivo: 0160.vital
Procesando archivo: 0163.vital
Procesando archivo: 0167.vital
Procesan

In [146]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1075328 entries, 0 to 1075327
Data columns (total 13 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Solar8000/ETCO2       1007385 non-null  float64
 1   Solar8000/FEO2        1007254 non-null  float64
 2   Solar8000/FIO2        1007351 non-null  float64
 3   Solar8000/HR          990694 non-null   float64
 4   Solar8000/INCO2       1007477 non-null  float64
 5   Solar8000/RR_CO2      960686 non-null   float64
 6   Solar8000/PLETH_SPO2  1015257 non-null  float64
 7   Solar8000/PLETH_HR    1014730 non-null  float64
 8   Solar8000/VENT_PIP    919531 non-null   float64
 9   Solar8000/VENT_TV     937441 non-null   float64
 10  Solar8000/VENT_RR     942599 non-null   float64
 11  Primus/MAC            1009964 non-null  float64
 12  caseid                1075328 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 106.7 MB


In [86]:
#Quiero consultar valores faltantes
# 2. VALORES FALTANTES
print("\n2. ANÁLISIS DE VALORES FALTANTES")
print("-" * 40)
missing_analysis3 = final_df.isnull().sum()
missing_pct3 = (missing_analysis3 / len(final_df)) * 100
missing_df3 = pd.DataFrame({
        'Missing_Count': missing_analysis3,
        'Missing_Pct': missing_pct3
    }).sort_values('Missing_Pct', ascending=False)
    
print("Columnas con valores faltantes:")
print(missing_df3[missing_df3['Missing_Count'] > 0])


2. ANÁLISIS DE VALORES FALTANTES
----------------------------------------
Columnas con valores faltantes:
                      Missing_Count  Missing_Pct
Solar8000/VENT_PIP           155797    14.488324
Solar8000/VENT_TV            137887    12.822785
Solar8000/VENT_RR            132729    12.343118
Solar8000/RR_CO2             114642    10.661119
Solar8000/HR                  84634     7.870529
Solar8000/FEO2                68074     6.330534
Solar8000/FIO2                67977     6.321513
Solar8000/ETCO2               67943     6.318351
Solar8000/INCO2               67851     6.309796
Primus/MAC                    65364     6.078517
Solar8000/PLETH_HR            60598     5.635304
Solar8000/PLETH_SPO2          60071     5.586296


Los porcentajes de valores nulos no son tal altos. Decido que la forma de abordarlos no es eliminando registros, pues podría perder registros consecutivos y alterar la veracidad de mis datos. Voy a  imputarlos con el método de último valor observado. que ya probé con la tabla exploratoria.

Primero observo estadísticas, antes de la imputación, para comparar luego de imputar datos nulos

In [87]:
#Observo estadísticas antes de la imputación para comparar luego de imputar datos nulos
# Estadísticas de ETCO2
print(f"\nEstadísticas de ETCO2:")
print(final_df['Solar8000/ETCO2'].describe())

# Estadísticas de FEO2
print(f"\nEstadísticas de FEO2:")
print(final_df['Solar8000/FEO2'].describe())

# Estadísticas de FIO2
print(f"\nEstadísticas de FIO2:")
print(final_df['Solar8000/FIO2'].describe())

# Estadísticas de HR
print(f"\nEstadísticas de HR:")
print(final_df['Solar8000/HR'].describe())

# Estadísticas de INCO2
print(f"\nEstadísticas de INCO2:")
print(final_df['Solar8000/INCO2'].describe())

# Estadísticas de RR_CO2
print(f"\nEstadísticas de RR_CO2:")
print(final_df['Solar8000/RR_CO2'].describe())

# Estadísticas de VENT RR
print(f"\nEstadísticas de VENT RR:")
print(final_df['Solar8000/VENT_RR'].describe())

# Estadísticas de PLETH_SPO2
print(f"\nEstadísticas de PLETH_SPO2:")
print(final_df['Solar8000/PLETH_SPO2'].describe())

# Estadísticas de PLETH_HR
print(f"\nEstadísticas de PLETH_HR:")
print(final_df['Solar8000/PLETH_HR'].describe())

# Estadísticas de VENT PIP
print(f"\nEstadísticas de VENT PIP:")
print(final_df['Solar8000/VENT_PIP'].describe())

# Estadísticas de VENT TV
print(f"\nEstadísticas de VENT TV:")
print(final_df['Solar8000/VENT_TV'].describe())

# Estadísticas de MAC
print(f"\nEstadísticas de MAC:")
print(final_df['Primus/MAC'].describe())

# Estadísticas de casos
print(f"\nEstadísticas de casos:")
print(final_df['caseid'].describe())


Estadísticas de ETCO2:
count    1.007385e+06
mean     3.307931e+01
std      8.701448e+00
min      0.000000e+00
25%      3.200000e+01
50%      3.500000e+01
75%      3.700000e+01
max      8.700000e+01
Name: Solar8000/ETCO2, dtype: float64

Estadísticas de FEO2:
count    1.007254e+06
mean     6.602261e+01
std      1.874778e+01
min      1.100000e+01
25%      5.100000e+01
50%      7.000000e+01
75%      8.400000e+01
max      1.000000e+02
Name: Solar8000/FEO2, dtype: float64

Estadísticas de FIO2:
count    1.007351e+06
mean     7.178781e+01
std      1.893795e+01
min      1.200000e+01
25%      5.600000e+01
50%      7.500000e+01
75%      9.100000e+01
max      1.000000e+02
Name: Solar8000/FIO2, dtype: float64

Estadísticas de HR:
count    990694.000000
mean         72.209031
std          16.865262
min           0.000000
25%          61.000000
50%          71.000000
75%          82.000000
max         259.000000
Name: Solar8000/HR, dtype: float64

Estadísticas de INCO2:
count    1.007477e+06
mean

In [149]:
# IMPUTACIÓN DE DATOS NULOS
# Puedo especificar un límite para la imputación para evitar que se propague por huecos de 
# tiempo muy largos.elijo llenar un máximo de 5 valores nulos consecutiv
df_imputado = final_df.ffill(limit=5)

# Rellenar los valores nulos al inicio de la serie
df_imputado_completo = df_imputado.ffill().bfill()

df_imputado_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1075328 entries, 0 to 1075327
Data columns (total 13 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Solar8000/ETCO2       1075328 non-null  float64
 1   Solar8000/FEO2        1075328 non-null  float64
 2   Solar8000/FIO2        1075328 non-null  float64
 3   Solar8000/HR          1075328 non-null  float64
 4   Solar8000/INCO2       1075328 non-null  float64
 5   Solar8000/RR_CO2      1075328 non-null  float64
 6   Solar8000/PLETH_SPO2  1075328 non-null  float64
 7   Solar8000/PLETH_HR    1075328 non-null  float64
 8   Solar8000/VENT_PIP    1075328 non-null  float64
 9   Solar8000/VENT_TV     1075328 non-null  float64
 10  Solar8000/VENT_RR     1075328 non-null  float64
 11  Primus/MAC            1075328 non-null  float64
 12  caseid                1075328 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 106.7 MB


Puedo observar que la totalidad de las columnas quedaron sin datos nulos.

In [ ]:
#Observo estadísticas después de la imputación para comparar luego de imputar datos nulos
# Estadísticas de ETCO2
print(f"\nEstadísticas de ETCO2:")
print(df_imputado_completo['Solar8000/ETCO2'].describe())

# Estadísticas de FEO2
print(f"\nEstadísticas de FEO2:")
print(df_imputado_completo['Solar8000/FEO2'].describe())

# Estadísticas de FIO2
print(f"\nEstadísticas de FIO2:")
print(df_imputado_completo['Solar8000/FIO2'].describe())

# Estadísticas de HR
print(f"\nEstadísticas de HR:")
print(df_imputado_completo['Solar8000/HR'].describe())

# Estadísticas de INCO2
print(f"\nEstadísticas de INCO2:")
print(df_imputado_completo['Solar8000/INCO2'].describe())

# Estadísticas de RR_CO2
print(f"\nEstadísticas de RR_CO2:")
print(df_imputado_completo['Solar8000/RR_CO2'].describe())

# Estadísticas de VENT RR
print(f"\nEstadísticas de VENT RR:")
print(df_imputado_completo['Solar8000/VENT_RR'].describe())

# Estadísticas de PLETH_SPO2
print(f"\nEstadísticas de PLETH_SPO2:")
print(df_imputado_completo['Solar8000/PLETH_SPO2'].describe())

# Estadísticas de PLETH_HR
print(f"\nEstadísticas de PLETH_HR:")
print(df_imputado_completo['Solar8000/PLETH_HR'].describe())

# Estadísticas de VENT PIP
print(f"\nEstadísticas de VENT PIP:")
print(df_imputado_completo['Solar8000/VENT_PIP'].describe())

# Estadísticas de VENT TV
print(f"\nEstadísticas de VENT TV:")
print(df_imputado_completo['Solar8000/VENT_TV'].describe())

# Estadísticas de MAC
print(f"\nEstadísticas de MAC:")
print(df_imputado_completo['Primus/MAC'].describe())

# Estadísticas de casos
print(f"\nEstadísticas de casos:")
print(df_imputado_completo['caseid'].describe())


Estadísticas de ETCO2:
count    1.075328e+06
mean     3.209797e+01
std      1.013698e+01
min      0.000000e+00
25%      3.200000e+01
50%      3.400000e+01
75%      3.700000e+01
max      8.700000e+01
Name: Solar8000/ETCO2, dtype: float64

Estadísticas de FEO2:
count    1.075328e+06
mean     6.542555e+01
std      1.946535e+01
min      1.100000e+01
25%      5.000000e+01
50%      6.900000e+01
75%      8.400000e+01
max      1.000000e+02
Name: Solar8000/FEO2, dtype: float64

Estadísticas de FIO2:
count    1.075328e+06
mean     7.147395e+01
std      2.000520e+01
min      1.200000e+01
25%      5.600000e+01
50%      7.500000e+01
75%      9.100000e+01
max      1.000000e+02
Name: Solar8000/FIO2, dtype: float64

Estadísticas de HR:
count    1.075328e+06
mean     7.284427e+01
std      1.762043e+01
min      0.000000e+00
25%      6.200000e+01
50%      7.100000e+01
75%      8.200000e+01
max      2.590000e+02
Name: Solar8000/HR, dtype: float64

Estadísticas de INCO2:
count    1.075328e+06
mean     8.4

Al comparar estadísticas puedo notar que luego de la imputacion los valores promedio de casi todas las variable son un poco menores, pero en cantidades mínimas, de lo que deduzco que la imputación de datos no afecta de manera crítica ni significativa el contenido de mi data.

-->> Voy a exportar los archivos resultantes hasta ahora: cirugias_VUP_filtrado y df_imputado_completo a formato CSV.

In [91]:
# Exportar el dataset limpio que tengo hasta este momento, en formato CSV
cirugias_VUP_filtrado2.to_csv('cirugias_VUP2.csv', index=False)
# Exportar el dataset limpio que tengo hasta este momento, en formato CSV
df_imputado_completo.to_csv('vitals2.csv', index=False)

## Combinación de las tablas Cirguías (información estática) + Vitals (series de tiempo)

In [150]:
# Realizar la combinación de los dos DataFrames
datos_completos = pd.merge(df_imputado_completo, cirugias_VUP_filtrado2, on='caseid', how='left')

# Mostrar la nueva estructura del DataFrame
print("Estructura del DataFrame combinado:")
datos_completos.info()

Estructura del DataFrame combinado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1075328 entries, 0 to 1075327
Data columns (total 48 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Solar8000/ETCO2       1075328 non-null  float64
 1   Solar8000/FEO2        1075328 non-null  float64
 2   Solar8000/FIO2        1075328 non-null  float64
 3   Solar8000/HR          1075328 non-null  float64
 4   Solar8000/INCO2       1075328 non-null  float64
 5   Solar8000/RR_CO2      1075328 non-null  float64
 6   Solar8000/PLETH_SPO2  1075328 non-null  float64
 7   Solar8000/PLETH_HR    1075328 non-null  float64
 8   Solar8000/VENT_PIP    1075328 non-null  float64
 9   Solar8000/VENT_TV     1075328 non-null  float64
 10  Solar8000/VENT_RR     1075328 non-null  float64
 11  Primus/MAC            1075328 non-null  float64
 12  caseid                1075328 non-null  int64  
 13  casestart             1075328 non-null  int64  
 14

In [152]:
#Almaceno los datos completos antes de empezar a preparar datos para modelos
datos_completos.to_csv('dataset_fuente_completo2.csv', index=False)